In [99]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
df = pd.read_csv("./SRZWebAnaluticsByModules_2018-11-06.csv",sep=",")
df1 = pd.read_csv("./SRZWebAnaluticsByModules_2018-11-07.csv",sep=",")
# df=df.append(df1)
engine = create_engine('sqlite:///mydatabase.db', echo=False)

def ConvertTime(time): 
    if time[-2:]=='AM' and time[:2]=='12': 
        return '00' + time[2:-2]
    elif time[-2:]=='AM': 
        return '00' + time[2:-2]
    elif time[-2:]=='PM' and time[:2]=='12':
        return time[0:-2]
    else:
        return str(int(time[:2])+12)+time[2:-2]
#print(ConvertTime('11:59:29 PM'))


def RemoveTrash(login):
    return login.replace('i:0#.w|','')
#print(RemoveTrash(' i:0#.w|spb99-uscpfarmsvc'))


conn = sqlite3.connect("mydatabase.db") # или :memory: чтобы сохранить в RAM
cursor = conn.cursor()
 
# Создание таблицы
# cursor.execute("""CREATE TABLE Visits
#                   (UserLogin text, ServerUrl text, SiteUrl text,
#                    WebUrl text, DocumentPath text, ComingDate date, ComingTime DATETIME )
#                """)
# 
# conn.commit()
# cursor.execute("""drop TABLE Visits
#                """)
# 
# conn.commit()

dfTemp = pd.DataFrame(df['LogTime'].str.split(' ',1).tolist(),columns=['ComingDate','ComingTime'])
df=df.drop(['LogTime'],axis=1)
dfTemp['ComingTime'] = dfTemp['ComingTime'].apply(lambda x: ConvertTime(x.rjust(11,'0')))
dfTemp['ComingDate'] = dfTemp['ComingDate'].apply(lambda x: x.replace('/','.'))
df['UserLogin'] = df['UserLogin'].apply(lambda x: RemoveTrash(x))

df = pd.concat([df,dfTemp],axis=1)
dfTemp = pd.DataFrame(df1['LogTime'].str.split(' ',1).tolist(),columns=['ComingDate','ComingTime'])
df1=df1.drop(['LogTime'],axis=1)
dfTemp['ComingTime'] = dfTemp['ComingTime'].apply(lambda x: ConvertTime(x.rjust(11,'0')))
dfTemp['ComingDate'] = dfTemp['ComingDate'].apply(lambda x: x.replace('/','.'))
df1['UserLogin'] = df1['UserLogin'].apply(lambda x: RemoveTrash(x))
df1 = pd.concat([df1,dfTemp],axis=1)
# 
# df.to_sql('Visits', con=conn)
df1.to_sql('Visits', con=conn,if_exists='append')
# conn.commit()
print(df1)


                UserLogin         ServerUrl SiteUrl WebUrl  \
0         tekhnolog.cdng2  http://srz.local       /    NaN   
1         tekhnolog.cdng2  http://srz.local       /    NaN   
2           brusentsev.ob  http://srz.local     NaN    NaN   
3           brusentsev.ob  http://srz.local     NaN    NaN   
4            belozerov.bv  http://srz.local     NaN    NaN   
...                   ...               ...     ...    ...   
215389           lanin.vo  http://srz.local     NaN    NaN   
215390  nt authority\iusr  http://srz.local     NaN    NaN   
215391           lanin.vo  http://srz.local     NaN    NaN   
215392  nt authority\iusr  http://srz.local     NaN    NaN   
215393           lanin.vo  http://srz.local     NaN    NaN   

                                             DocumentPath ComingDate  \
0       /_layouts/15/iteco.srz.sps2013.common.analytic...  11.7.2018   
1       /_layouts/15/iteco.srz.sps2013.common.analytic...  11.7.2018   
2                                      